"Code trouble": when iteration is occuring inside excel file, the stopping criterion is:
        while xls_sheet.cell_value(i, 1) != '': 
Meaning that if there no BB angle associate with this line, the loop stop. The problem is that sometimes (not often) there exists centriole with coordinate but no angle.. Leading to an early stop iteration in the xls file.

#### Import packages

In [1]:
# Image processing
from PIL import Image 

# for working on excel file
import xlrd  

# To get a list of the file in a folder
import glob  

import numpy as np

In [2]:
cd C:\Users\breto\Desktop\Data_Analysis\Schmidtea_CNN\

C:\Users\breto\Desktop\Data_Analysis\Schmidtea_CNN


In [6]:
# List all .xlsm files (contain BB coordinate and orientation) in the folder 'data'
xls_list = glob.glob("C:/Users/breto/Desktop/Data_Analysis/Schmidtea_CNN/data_validation/*.xlsm")

# List all .tif images (where BB are annotated) in the folder 'data'
tif_list = glob.glob("C:/Users/breto/Desktop/Data_Analysis/Schmidtea_CNN/data_validation/*.tif")

In [7]:
xls_list

['C:/Users/breto/Desktop/Data_Analysis/Schmidtea_CNN/data_validation\\170404_Ctrl_injection_18dpa_Tail_2 (V3-2c).xlsm']

In [8]:
tif_list

['C:/Users/breto/Desktop/Data_Analysis/Schmidtea_CNN/data_validation\\170404_Ctrl_injection_18dpa_Tail_2 (V3-2c).tif']

In [10]:
if len(xls_list) > len(tif_list):
    print("Take care: an Xls file is missing")
elif len(xls_list) < len(tif_list):
    print("Take care: an Tif file is missing")

for i in range(max(len(xls_list), len(tif_list))):
    if xls_list[i][:-4] != tif_list[i][:-3]:
        print("A xls/tif is not appariate with a tif/xsl")

## Define size of the crop

In [11]:
#Size of the expected image output
size_img = 32

### Extract images

In [12]:
# Define Image size openable with PIL (default size is too small)
Image.MAX_IMAGE_PIXELS = 10000000000

# Initialisation of the centriole (~Basal body) coordinate and angle list
count = 0
centriole_coordinate = []
angle_list = []
centriole_img_list = []

# Iteration trough the excel file
for k in range(len(xls_list)):
    print("Number of file treated: {}/{}".format(k+1, len(xls_list)))

    # Open the image
    tif_file = Image.open(tif_list[k])
    
    # Open the excel file and go to appropriate sheet (index 0)
    xls_file = xlrd.open_workbook(xls_list[k])
    xls_sheet = xls_file.sheet_by_index(0)

    # In the Excel file, the coordinate of BB starts at row 7
    # This variable will iterate a reinitialisation is required after each xls file
    i = 7

    # In the Excel file, iterate to find coordinate of all the centrioles
    while xls_sheet.cell_value(i, 1) != '':
        # Recover parameter of a centriol
            # count = centriole number (ID)
            # xls_(i,3) = X coordinate of the centriole
            # xls_(i,4) = Y coordinate of the centriole
            # xls_(i,1) = Orienation (Angle) of the centriole
        coord = [count, xls_sheet.cell_value(i, 3), xls_sheet.cell_value(i, 4), xls_sheet.cell_value(i, 1)]
        
        # Append the basal body to the list of all centrioles
        centriole_coordinate.append(coord)

        # Create a list of the angle only
        angle = [xls_sheet.cell_value(i, 1)]
        angle_list.append(angle)

        # Extract the centriole on the image
        centriole_cropped = tif_file.crop((centriole_coordinate[count][1] - size_img/2, centriole_coordinate[count][2] - size_img/2, centriole_coordinate[count][1] + size_img/2, centriole_coordinate[count][2] + size_img/2))
        #centriole_cropped.save("centriole_" + str(count) + ".tif")  # No need to save it
        centriole_img_array = np.asarray(centriole_cropped)
        centriole_img_list.append(centriole_img_array)
        i += 1
        count += 1

centriole_list_array = np.asarray(centriole_img_list, dtype="int32")
angle_list_array = np.asarray(angle_list)
angle_list_array = angle_list_array.ravel()


Number of file treated: 1/1


### Data Exploration andcleaning

**Same number of extracted images and angle?**

In [13]:
if centriole_list_array.shape[0] != angle_list_array.shape[0]:
    print("Take your phone and Call Sherlock Holmes: you have a problem")
else: 
    print("So far so good")
    print("\n")
    print("Centriole list shape: {}".format(centriole_list_array.shape))
    print("    Angle list shape: {}".format(angle_list_array.shape))

So far so good


Centriole list shape: (813, 32, 32)
    Angle list shape: (813,)


**Presence of Nan values?**

In [14]:
print("Nan value in extracted images: {}".format(np.count_nonzero(np.isnan(centriole_list_array))))
print("Nan value in extracted angles: {}".format(np.count_nonzero(np.isnan(angle_list_array))))

Nan value in extracted images: 0
Nan value in extracted angles: 0


**Absence of value in images?**

In [15]:
x = centriole_list_array

img_trouble_id =[]
for i in range(x.shape[0]):
    if x[i].min() == x[i].max():
        img_trouble_id.append(i)
        
del x

print("For {} images, the picture is completely black".format(len(img_trouble_id)))

For 0 images, the picture is completely black


Remove the problematic images.

Note: It could be interesting to look at the file to understand why this happens.

In [16]:
img_to_delete = np.asarray(img_trouble_id)
clean_centriole_array = centriole_list_array
clean_angle_array = angle_list_array

In [17]:
j=0
for i in reversed(img_to_delete):
    j += 1
    if j % 100 == 0 :
        print("Treated images: {}/{}".format(j, img_to_delete.shape[0]))
    clean_centriole_array = np.delete(clean_centriole_array, i, 0)
    clean_angle_array = np.delete(clean_angle_array, i, 0)

**Check that the modification fix everything**

In [18]:
x = clean_centriole_array

img_trouble_id =[]
for i in range(x.shape[0]):
    if x[i].min() == x[i].max():
        img_trouble_id.append(i)
        
del x

if len(img_trouble_id) == 0:
    print("So far, so good")
    print("\n")
    print("Centriole list shape: {}".format(clean_centriole_array.shape))
    print("    Angle list shape: {}".format(clean_angle_array.shape))
else: 
    print("For {} images, the picture is completely black".format(len(img_trouble_id)))

So far, so good


Centriole list shape: (813, 32, 32)
    Angle list shape: (813,)


### Save Data

In [19]:
# save as numpy
np.save('centriole_angle_32pixels_validation', clean_angle_array)
np.save('centriole_img_32pixels_validation', clean_centriole_array)